In [44]:
whorunsit = 'LiezelMac' # 'LiezelMac' | 'LiezelCluster'

if whorunsit == 'LiezelMac':
    home_dir = '/Users/ltamon'
elif whorunsit == 'LiezelCluster':
    home_dir = '/project/sahakyanlab/ltamon'
else:
  print("The supplied <whorunsit> option is not created in the script.")

wk_dir = home_dir + '/SahakyanLab/GenomicContactDynamics/26_PredictingCp'
csv_dir = wk_dir + '/out_makeTable'
out_dir = wk_dir + '/out_autogluon_run'

gcb = 'min2Mb'
chrs = ['chr21']

model_id = 'compl_kmer_meanGC'
feat_regex = 'grp.compl.'
label_col = 'label'
subsample_size = 50  # subsample subset of data for quick run

model_path = out_dir + '/' + 'agModel_'  + model_id
metric = 'roc_auc'

# fit()
time_limit = 2*60 # 2 min
#holdout_frac

# Baggin/stack ensembling - if enabled, dont provide tuning_data
#num_bag_folds=5 # how many times the k-fold bagging process is repeated to further reduce variance (increasing this may further boost accuracy but will substantially increase training times, inference latency, and memory/disk usage)
#num_bag_sets=1, 
#num_stack_levels=1 
auto_stack=True # Autogluon will select bagging/stacking numbers
# specifying presets='best_quality' in fit() simply sets auto_stack=True
    
#num_trials = 5  # try at most 5 different hyperparameter configurations for each type of model
#search_strategy = 'auto'  # to tune hyperparameters using random search routine with a local scheduler
#hyperparameter_tune_kwargs = {  # HPO is not performed unless hyperparameter_tune_kwargs is specified
#    'num_trials': num_trials,
#    'scheduler' : 'local',
#    'searcher': search_strategy,
#}

In [30]:
import pandas as pd
import re
import numpy as np
from autogluon.tabular import TabularDataset, TabularPredictor

In [31]:
def concat_chrcsv(csv_dir, gcb, chrs):
    
    df = []
    [df.append(pd.read_csv( csv_dir + '/' + gcb + '_' + chr + '_MLtbl.csv' )) for chr in chrs]
    df = pd.concat(df)
    
    return(df)

def choose_features(df, feat_regex, label_col):
    
    final_regex = feat_regex + '|' + label_col
    is_chosen = [ bool(re.search(pattern=final_regex, string=feat)) for feat in df.columns ] 
    is_todrop = [not bool for bool in is_chosen]
    df.drop(columns=list(df.columns[is_todrop]), inplace=True)

    return(df)
    
def switch_contact(df):
    
    features = np.array(df.columns)
    is_icol = ['.i_' in feat for feat in features]
    is_jcol = ['.j_' in feat for feat in features]
    
    features_switch = np.array(features)
    features_switch[is_icol] = features[is_jcol]
    features_switch[is_jcol] = features[is_icol]
    
    df_switch = df.set_axis(features_switch, axis=1, inplace=False)
    df_switch = df_switch[features]
    df = pd.concat([df, df_switch])
    
    return(df)

In [34]:
train_data = concat_chrcsv(csv_dir, gcb, chrs)

In [35]:
train_data = choose_features(train_data, feat_regex, label_col)

In [36]:
train_data = switch_contact(train_data)

In [39]:
train_data = TabularDataset(train_data)

<bound method NDFrame.head of         label  grp.compl.ij_anvdist  grp.compl.ij_CIIkmer  grp.compl.ij_CIIG  \
0           6           2234.495310               -1.2627          -0.000020   
1           8           2577.978790               -1.2975          -0.000020   
2           9           1756.299497               -1.4243          -0.000019   
3           4           3603.515907               -1.5271          -0.000020   
4           8           4365.757094               -1.4998          -0.000020   
...       ...                   ...                   ...                ...   
311966     10           4795.648879               -1.3605          -0.000023   
311967     16           3313.459469               -1.1830          -0.000024   
311968     13           4838.429222               -1.4811          -0.000022   
311969     13           3636.950564               -1.0367          -0.000025   
311970     12           2854.515797               -1.0911          -0.000024   

        g

In [4]:
train_data = train_data.sample(n=subsample_size, random_state=0)

,label,compl.CII_kmer,compl.CII_G,compl.CII_align,anv.distances,anv.i_nA,anv.i_nC,anv.i_nG,anv.i_nT,anv.i_zetaA,...,kmer3.j_GTA,kmer3.j_TAA,kmer3.j_TCA,kmer1.i_A,kmer1.i_C,kmer1.j_A,kmer1.j_C,GC.i_GC,GC.j_GC,GC.meanij_GC
289282,1,-1.6786,-0.000019,-0.544575,6944.867223,13358,7081,7360,12201,20223.190298,...,557,863,1380,25559,14441,18918,21082,0.361025,0.527050,0.444037
21376,11,-1.0162,-0.000022,-0.518800,2549.100760,12012,7986,7923,12079,19733.248085,...,1006,2301,1621,24091,15909,26212,13788,0.397725,0.344700,0.371213
47990,7,-0.9191,-0.000023,-0.516975,1835.851846,13311,7381,7434,11874,20349.076703,...,932,1819,1654,25185,14815,24784,15216,0.370375,0.380400,0.375387
238110,14,-1.0989,-0.000024,-0.526250,2938.775774,10240,8741,8995,12024,19673.823437,...,746,1211,1432,22264,17736,21102,18898,0.443400,0.472450,0.457925
89,5,-1.2689,-0.000020,-0.514575,3916.799612,12868,6779,6868,13485,18489.428660,...,875,1493,1613,26353,13647,24357,15643,0.341175,0.391075,0.366125


In [11]:
#from autogluon.features.generators import AutoMLPipelineFeatureGenerator
#predictor = TabularPredictor(label='label')
#predictor.fit(train_data, hyperparameters={'GBM' : {}}, feature_generator=AutoMLPipelineFeatureGenerator())

In [ ]:
# dfx["B"] = dfx["B"].astype("category") # To specify column type

In [ ]:
predictor = TabularPredictor(label=label_col, eval_metric=metric, path=model_path).fit(train_data, time_limit=time_limit)

In [ ]:
results = predictor.fit_summary(show_plot=True)

In [29]:
predictor = TabularPredictor.load(model_path)  # `predictor.path` is another way to get the relative path needed to later load predictor.